In [111]:
import numpy as np

DATA_DIR = 'F:\\technopolis\\data\\'
GRAPH_PATH = 'F:\\technopolis\\graph\\'
MODEL_DIR = 'F:\\technopolis\\model\\'
SAMPLE_FRAC = 0.01
BATCH_SIZE = 1_000_000

SVM_ALPHA_RANGE = np.fromfunction(lambda i: 1e-15 * pow(10, i), (15,), dtype=float)
SVM_MAX_ITER_RANGE = np.fromfunction(lambda i: 10 * (i + 1), (20,), dtype=float)

FEATURES = [
    'DeviceClass',
    'DeviceName',
    'DeviceBrand',
    'DeviceCpu',
    'DeviceCpuBits',
    'OperatingSystemClass',
    'OperatingSystemName',
    'OperatingSystemVersion',
    'OperatingSystemNameVersion',
    'OperatingSystemVersionBuild',
    'LayoutEngineClass',
    'LayoutEngineName',
    'LayoutEngineVersion',
    'LayoutEngineVersionMajor',
    'LayoutEngineNameVersion',
    'LayoutEngineNameVersionMajor',
    'AgentClass',
    'AgentName',
    'AgentVersion',
    'AgentVersionMajor',
    'AgentNameVersion',
    'AgentNameVersionMajor',
#     'from',
#     'to',
#     'url',
#     'requestType',
    'operation'
]

In [112]:
!pip install pyarrow
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import seaborn as sns
import user_agents
%matplotlib inline
from matplotlib import pyplot as plt
pd.set_option("display.precision", 2)

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.tree import export_graphviz
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.externals import joblib
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel

import os

import warnings
warnings.filterwarnings('ignore')

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## Выбор фич

In [131]:
http = pq.read_table(DATA_DIR + 'HTTPRequests-20180217_1718_parsedUAA_parquet.parquet', columns=FEATURES ).to_pandas().sample(frac=SAMPLE_FRAC)
target = pq.read_table(DATA_DIR + 'HTTPRequests-20180217_1718_parsedUAA_parquet.parquet', columns=['isBot'] ).to_pandas().sample(frac=SAMPLE_FRAC)

In [132]:
# убираем None-ы
for f in FEATURES:
    http[f] = http[f].astype('str').fillna(f+'None')
    http[f] = f + ';' + http[f].astype('str')
    
# получаем все значения фич в виде матрицы
column_to_arrays = http.as_matrix(columns=FEATURES).transpose()
array_to_encode = column_to_arrays[0]

# объединяем в один np массив, чтобы прогнать в le
for i in range(1, len(column_to_arrays)):
    array_to_encode = np.concatenate((array_to_encode, column_to_arrays[i]), axis=0)

le = LabelEncoder()
le.fit(array_to_encode)

LabelEncoder()

In [133]:
# применяем le к значениям фич
X_le = http[FEATURES].apply(le.transform, axis=0)
y = target

In [134]:
ohe = OneHotEncoder(handle_unknown='ignore')
X_ohe = ohe.fit_transform(X_le)

In [135]:
clf = SGDClassifier(random_state=17, max_iter=1000, alpha=0.0000001, loss='log')
feature_selector = SelectFromModel(clf)

print("Shape before feature selection: {}".format(X_ohe.shape))
X_new = feature_selector.fit_transform(X_ohe, y)
print("Shape after feature selection: {}".format(X_new.shape))
print("Selected:")
selected = feature_selector.get_support(indices=True)
for f in selected:
    try:
        print(le.inverse_transform([f])[0])
    except ValueError:
        pass


Shape before feature selection: (10104, 2136)
Shape after feature selection: (10104, 523)
Selected:
AgentClass;1.0
AgentClass;1.5.1106.241
AgentClass;11.0
AgentClass;11.0.3
AgentClass;14.10.2062.12521
AgentClass;17.11.1.628.01
AgentClass;18.0.1290.67495
AgentClass;32.0.1725.115
AgentClass;35.0.2070.100283
AgentClass;36.0.2130.46
AgentClass;39.0.2171.95
AgentClass;40.0.2308.62
AgentClass;40.0.2308.81
AgentClass;41.0.2353.46
AgentClass;43.0.2442.1192
AgentClass;44.0.2403.119
AgentClass;44.0.2403.125
AgentClass;44.0.2403.131
AgentClass;44.0.2403.157
AgentClass;44.4.2403.3
AgentClass;45.0.2246.125120
AgentClass;45.0.2454.101
AgentClass;45.0.2454.112
AgentClass;45.0.2552.882
AgentClass;45.0.2552.888
AgentClass;46.0.2490.76
AgentClass;47.5.2526.115
AgentClass;47.5.2526.173
AgentClass;48.0.2564.103
AgentClass;48.0.2564.116
AgentClass;49.0.2623.112
AgentClass;54.0.2840.98
AgentClass;54.0.2840.99
AgentClass;55.0.2883.87
AgentClass;55.0.2883.91
AgentClass;55.0.2883.95
AgentClass;56.0.2924.197
Ag